# Multiple Species Weighting Function Calculations

One of the most useful features of SK-DO is that it can compute weighting functions with respect to any number of input atmospheric species as well as surface reflectance.

In [1]:
%matplotlib inline

In [2]:
import sasktran as sk
import sasktran.disco.interface as do
import numpy as np
import matplotlib.pyplot as plt

Start by setting up the geometry, use a SZA of 60 degrees with a pure nadir viewing geometry.  Set up the atmosphere to contain rayleigh scattering, ozone and NO2 absorption, and a Lambertian surface

In [3]:
geometry = sk.NadirGeometry()
geometry.from_zeniths_and_azimuth_difference(60, 0, 0)

atmosphere = sk.Atmosphere()

# add our species
atmosphere['rayleigh'] = sk.Species(sk.Rayleigh(), sk.MSIS90())
atmosphere['o3'] = sk.Species(sk.O3OSIRISRes(), sk.Labow())
atmosphere['no2'] = sk.Species(sk.NO2OSIRISRes(), sk.Pratmo())
atmosphere.atmospheric_state = sk.MSIS90()

# add our surface properties
# setting to a scalar automatically sets the surface to be Lambertian
atmosphere.brdf = 0.3

Next we set the weighting function calculation to calculate with respect to all three species in the atmosphere as well as the surface

In [4]:
atmosphere.wf_species = ['rayleigh', 'o3', 'no2', 'brdf']

Run the calculation and print the results

In [5]:
wavelengths = np.linspace(340,700, 361)
engine = do.EngineDO(geometry=geometry, atmosphere=atmosphere, wavelengths=wavelengths)
rad = engine.calculate_radiance()
print(rad)

<xarray.Dataset>
Dimensions:            (wavelength: 361, los: 1, xyz: 3, perturbation: 201)
Coordinates:
  * wavelength         (wavelength) float64 340.0 341.0 342.0 ... 699.0 700.0
  * xyz                (xyz) <U1 'x' 'y' 'z'
  * perturbation       (perturbation) object MultiIndex
  * altitude           (perturbation) int64 0 500 1000 ... 99000 99500 100000
Dimensions without coordinates: los
Data variables:
    radiance           (wavelength, los) float64 0.06624 0.06641 ... 0.04759
    mjd                (los) float64 5.82e+04
    los_vector         (los, xyz) float64 0.1761 0.588 -0.7894
    observer_position  (los, xyz) float64 -1.231e+06 -4.111e+06 5.486e+06
    wf_brdf            (wavelength, los) float64 0.085 0.08612 ... 0.1511 0.1512
    wf_rayleigh        (wavelength, los, perturbation) float64 2.253e-23 ... ...
    wf_o3              (wavelength, los, perturbation) float64 -3.675e-18 ......
    wf_no2             (wavelength, los, perturbation) float64 -9.505e-16 ......
 

In the resulting dataset we have weighting functions for the three species as a function of altitude, and the weighting function for the surface that is not a function of altitude.